In [14]:
from core.utils import Tibanna
from core import ff_utils
from datetime import datetime

# set enviroment and key/connection
env = 'fourfront-webdev'
tibanna = Tibanna(env=env)
ff = ff_utils.fdn_connection(key=tibanna.ff_keys)

def get_wfr_report(wfr):
    """For a given workflow_run_sbg item, grabs details, uuid, run_status, wfr name, date, and run time"""
    wfr_data= ff_utils.get_metadata(wfr , connection=ff)
    wfr_uuid = wfr_data['uuid']
    wfr_status = wfr_data['run_status']
    wfr_name = wfr_data['title'].split(' run ')[0]
    wfr_time = datetime.strptime(wfr_data['date_created'],'%Y-%m-%dT%H:%M:%S.%f+00:00')
    run_hours = (datetime.now()-wfr_time).total_seconds()/3600
    wfr_name_list = wfr_data['title'].split(' run ')[0].split('/')
    wfr_name = wfr_name_list[0]
    try:
        wfr_rev = wfr_name_list[1] 
    except:
        wfr_rev = "0"
    wfr_rep = {'wfr_uuid': wfr_data['uuid'],
               'wfr_status': wfr_data['run_status'],
               'wfr_name': wfr_name,
               'wfr_rev': wfr_rev,
               'wfr_date': wfr_time,
               'run_time': run_hours}
    return wfr_rep

    
def printTable(myDict, colList=None):
    """ Pretty print a list of dictionaries Author: Thierry Husson"""
    if not colList: colList = list(myDict[0].keys() if myDict else [])
    myList = [colList] # 1st row = header
    for item in myDict: myList.append([str(item[col] or '') for col in colList])
    colSize = [max(map(len,col)) for col in zip(*myList)]
    formatStr = ' | '.join(["{{:<{}}}".format(i) for i in colSize])
    myList.insert(1, ['-' * i for i in colSize]) # Seperating line
    for item in myList: print(formatStr.format(*item))   

In [15]:
list_wfr = ['3bb84eba-aded-41b1-8294-6801ff7831ed',
            'a3ad1c5c-a01e-41b0-8089-e0fef9aee0e4',
            'da7afb09-53e2-4650-9671-9fa3caf1952c',
            '33041c9e-b500-47ab-bf60-37aeb66c8150']
for wfr in list_wfr:
    print get_wfr_report(wfr)

{'wfr_status': u'complete', 'wfr_date': datetime.datetime(2017, 7, 18, 15, 15, 4, 313496), 'wfr_name': u'hi-c-processing-partb', 'run_time': 19.657053276944445, 'wfr_rev': u'28', 'wfr_uuid': u'3bb84eba-aded-41b1-8294-6801ff7831ed'}
{'wfr_status': u'complete', 'wfr_date': datetime.datetime(2017, 7, 16, 19, 39, 33, 891326), 'wfr_name': u'hi-c-processing-parta-juicer', 'run_time': 63.248908236666665, 'wfr_rev': u'25', 'wfr_uuid': u'a3ad1c5c-a01e-41b0-8089-e0fef9aee0e4'}
{'wfr_status': u'complete', 'wfr_date': datetime.datetime(2017, 6, 22, 18, 25, 33, 908169), 'wfr_name': u'md5', 'run_time': 640.4822996874999, 'wfr_rev': '0', 'wfr_uuid': u'da7afb09-53e2-4650-9671-9fa3caf1952c'}
{'wfr_status': u'running', 'wfr_date': datetime.datetime(2017, 6, 23, 16, 10, 18, 665551), 'wfr_name': u'fastqc-0-11-4-1', 'run_time': 618.7365975469444, 'wfr_rev': u'1', 'wfr_uuid': u'33041c9e-b500-47ab-bf60-37aeb66c8150'}


In [13]:
# This script is looking at the given enviroment and going through all the input files
# to see if any of then have a mising/deleted/obsolete input file.
# The first part is only reporting, and the second part is if you want to change the status
# to deleted and also delete output files.
# Give it some time before rerunning

delete_workflows = raw_input("Do you want to delete old workflowruns (if not, only report will be displayed (y/n))")

# what kind of files should be searched for worflow run inputs, use url compatible naming
raw_file_types = ['files-fastq']

# accepted workflows
workflow_details = [{'wf_name':'md5', 'revision':['0']},
                    {'wf_name':'fastqc-0-11-4-1', 'revision':['1']},
                    {'wf_name':'hi-c-processing-parta-juicer', 'revision':['25','26']},
                    {'wf_name':'hi-c-processing-partb', 'revision':['28']},
                    {'wf_name':'hi-c-processing-partc', 'revision':['2']}
                   ]
 
deleted_wfr_no = 0
files_with_deleted_wfr = 0
deleted_last_md5 = 0
deleted_last_fastqc = 0

# get all wfr
for raw_file_type in raw_file_types:
    # get all files for the given file type
    files = ff_utils.get_metadata(raw_file_type , connection=ff)['@graph']
    for raw_file in files:
        # switch for counting files with deleted workflow_runs
        deleted_wf = False
        wfr_report = []
        wfrs = raw_file.get('workflow_run_inputs')
        # get a report on all workflow_runs
        if wfrs:
            for wfr in wfrs:
                wfr_report.append(get_wfr_report(wfr))  
        # sort the report by date and name
        wfr_report = sorted(wfr_report, key=lambda k: (k['wfr_date'], k['wfr_name']))
        printTable(wfr_report)
        break
        
        
        #printTable(wfr_report,['wfr_date', 'wfr_name']) 
        for wf_name in workflow_names:
            #for each type of worklow make a list of old ones, and patch status and description
            sub_list_del = [i for i in wfr_report if i['wfr_name'] == wf_name][:-1]
            if sub_list_del:
                deleted_wf = True
                for wfr_to_del in sub_list_del:
                    deleted_wfr_no += 1
                    if delete_workflows.lower() in ['y', 'yes']:
                        patch_data = {'description': "This workflow run is deleted since there was a new run",
                                      'status': "deleted"}
                        ff_utils.patch_metadata(patch_data, obj_id=wfr_to_del['wfr_uuid'] ,connection=ff)
        
            #for each type of workflow check if the active one functional, if not, delete
            try:
                active_wfr = [i for i in wfr_report if i['wfr_name'] == wf_name][-1]
            except:
                continue
            if active_wfr['wfr_status'] != 'complete':
                if wf_name == 'md5':
                    # if it is more than 1 hour, it is not really running
                    if active_wfr['run_time']>1:
                        deleted_last_md5 += 1
                        if delete_workflows.lower() in ['y', 'yes']:
                            patch_data = {'status': "deleted"}
                            ff_utils.patch_metadata(patch_data, obj_id=active_wfr['wfr_uuid'] ,connection=ff)
                        
                if wf_name == 'fastqc-0-11-4-1/1':
                    # if it is more than 24 hours, it is not really running
                    if active_wfr['run_time']>24:
                        deleted_last_fastqc += 1 
                        if delete_workflows.lower() in ['y', 'yes']:
                            patch_data = {'status': "deleted"}
                            ff_utils.patch_metadata(patch_data, obj_id=active_wfr['wfr_uuid'] ,connection=ff)
              
            
        if deleted_wf:
            files_with_deleted_wfr += 1
        break

if delete_workflows.lower() in ['y', 'yes']:
    print str(deleted_wfr_no)," old workflowruns from", str(files_with_deleted_wfr), "files deleted"
    print str(deleted_last_md5), "md5 runs and", str(deleted_last_fastqc), "fastqc runs are problematic and deleted"
else:
    print str(deleted_wfr_no)," old workflowruns from", str(files_with_deleted_wfr), "files need to be deleted"
    print str(deleted_last_md5), "md5 runs and", str(deleted_last_fastqc), "fastqc runs are problematic and need to be deleted"        


Do you want to delete old workflowruns (if not, only report will be displayed (y/n))
wfr_status                    | wfr_date                   | wfr_name                     | run_time      | wfr_rev | wfr_uuid                            
----------------------------- | -------------------------- | ---------------------------- | ------------- | ------- | ------------------------------------
complete                      | 2017-05-12 21:58:12.262857 | md5                          | 1601.94532359 | 0       | 102e8aa3-8974-415d-93b5-ebd3439ea7d9
complete                      | 2017-05-15 21:43:58.245989 | fastqc-0-11-4-1              | 1530.18259124 | 1       | f7a23d37-c9fb-4a53-8527-9ea6bd8a949a
output_file_transfer_finished | 2017-06-01 20:10:14.323227 | hi-c-processing-parta-juicer | 1123.74483484 | 5       | f45d8d45-dc8f-47e6-b9c0-19952f1ea6b4
0  old workflowruns from 0 files need to be deleted
0 md5 runs and 0 fastqc runs are problematic and need to be deleted
